In [25]:
import requests
import csv
from datetime import datetime
import os
import pandas as pd
dir = 'NHTSA-FARS-download/'


# # Download from "make" API, which returns same data from 2010-2020 (except 2010 & 2011 had 2 fewer rows), and that pre-2010 gives errors.
year=2020
file = f'{dir}make{year}.csv'

urlMakes = f"https://crashviewer.nhtsa.dot.gov/CrashAPI/definitions/GetVariableAttributes?variable=make&caseYear={year}&format=csv"
r = requests.get(urlMakes, allow_redirects=True)
open(file, 'wb').write(r.content)

print(year, 'success', datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

makes = csv.reader( open(file,'r'))
next(makes)     #skip header row

for [makeID, make, _, _] in makes:
    # makes when iterated through:
    # ['id', 'text', 'from_year', 'to_year']
    # ['54', 'Acura', '1994', '']
    # ['31', 'Alfa Romeo', '1994', '']

    # models API is also cumulative in nature, so just final year needed.
    year=2020

    try:
        url = f"https://crashviewer.nhtsa.dot.gov/CrashAPI/definitions/GetVariableAttributesForModel?variable=model&caseYear={year}&make={makeID}&format=csv"
        rModels = requests.get(url, allow_redirects=True)

        # os.makedirs(dir+make, exist_ok = True)
        make = make.replace('/','-')

        with open( f"{dir}model-lookup/{makeID}-{make}_models{year}.csv", "w" ) as w:

            for i in range(len(my_list)):
                if i==0:
                    my_list[i][2] = 'makeID'
                else:
                    my_list[i][2] = makeID

                writer = csv.writer(w)
                writer.writerow(my_list[i])
        
        print('success:', datetime.now().strftime("%d/%m/%Y %H:%M:%S"), year, makeID, make)

    except Exception as e:
        print('FAILED:', datetime.now().strftime("%d/%m/%Y %H:%M:%S"), year, makeID, make, '\n', e)

    break

2020 success 18/04/2022 22:02:02
success: 18/04/2022 22:02:03 2020 54 Acura


In [3]:
print(rModels, rModels.content, sep='\n')

<Response [200]>
b'id,modelname,make\r\n37,CL,\r\n41,ILX,\r\n31,Integra,\r\n32,Legend,\r\n421,MDX,\r\n43,NSX (2016 on.  For 1991-2005 see model 033.),\r\n33,NSX (For 1991-2005 only.  For 2016 on see model 043.),\r\n398,Other (automobile),\r\n498,Other (light truck),\r\n402,RDX,\r\n36,RL/RLX,\r\n38,RSX,\r\n401,SLX,\r\n35,TL,\r\n44,TLX,\r\n39,TSX,\r\n999,Unknown (ACURA),\r\n399,Unknown (automobile),\r\n499,Unknown (light truck),\r\n34,Vigor,\r\n40,ZDX,\r\n'


In [26]:
decoded_content = rModels.content.decode('utf-8')
cr = csv.reader(decoded_content.splitlines(), delimiter=',')
my_list = list(cr)

print(type(cr))

with open( f"{dir}model-lookup/dropme.csv", "w" ) as w:

    for i in range(len(my_list)):
        if i==0:
            my_list[i][2] = 'makeID'
        else:
            my_list[i][2] = makeID
    
        writer = csv.writer(w)
        writer.writerow(my_list[i])




<class '_csv.reader'>


In [38]:
print(type(decoded_content), type(rModels.content.decode()))
print( list( csv.reader(decoded_content.splitlines())))












<class 'str'> <class 'str'>
[['id', 'modelname', 'make'], ['37', 'CL', ''], ['41', 'ILX', ''], ['31', 'Integra', ''], ['32', 'Legend', ''], ['421', 'MDX', ''], ['43', 'NSX (2016 on.  For 1991-2005 see model 033.)', ''], ['33', 'NSX (For 1991-2005 only.  For 2016 on see model 043.)', ''], ['398', 'Other (automobile)', ''], ['498', 'Other (light truck)', ''], ['402', 'RDX', ''], ['36', 'RL/RLX', ''], ['38', 'RSX', ''], ['401', 'SLX', ''], ['35', 'TL', ''], ['44', 'TLX', ''], ['39', 'TSX', ''], ['999', 'Unknown (ACURA)', ''], ['399', 'Unknown (automobile)', ''], ['499', 'Unknown (light truck)', ''], ['34', 'Vigor', ''], ['40', 'ZDX', '']]
